In [1]:
import mrpt
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

2024-03-31 22:48:21.203541: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-31 22:48:23.273070: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-31 22:48:23.276961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 22:48:26.975617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
name = 'mnist-784'
runs = 1
metric = 'euclidean'
queries = 1000
nameFull = name + '-' + metric + '-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

trainDataset :  (60000, 784)
testDataset :  (10000, 784)


***Create mrpt ann***

In [3]:
def createIndex(indexMethod, datasetImages):
    time_start = perf_counter()
    index = indexMethod(datasetImages)
    index.build_autotune_sample(0.8, 100)
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    return (index, totalTime)

In [4]:
(minBuildTime, maxBuildTime, indexedStruct) = createIndexNumerous(createIndex, mrpt.MRPTIndex, datasetTrainImages, runs)
print('minBuildTime : ', minBuildTime)
print('maxBuildTime : ', maxBuildTime)

index  1  created
minBuildTime :  4.104
maxBuildTime :  4.104


In [18]:
def measureTime(par, indexes, distances, datasetImages):
    totalTime = 0
    k = 100
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        index, distance = indexedStruct.ann(xq, return_distances=True)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        indexes.append(index[0])
        distances.append(distance[0])
    return np.round(totalTime, 3)

In [19]:
(minSearchTime, maxSearchTime, indexes, distances) = measureTimeNumerous(measureTime, runs, queries, datasetTestImages)
print('minSearchTime : ', minSearchTime)
print('maxSearchTime : ', maxSearchTime)

search  1  done
minSearchTime :  0.242
maxSearchTime :  0.242


In [20]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [22]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [23]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [24]:
amount = 10
compareElems(amount, indexes, distances, trueIndexes, trueDistances)

53843 || 53843
676.584 || 676.584
38620 || 38620
793.9868 || 793.9868
16186 || 16186
862.6766 || 862.6766
27059 || 27059
864.5039 || 864.5039
47003 || 47003
894.7 || 894.7
14563 || 14563
909.7043 || 909.7043
44566 || 44566
917.6323 || 917.6323
15260 || 15260
921.6241 || 921.6241
40368 || 40368
922.147 || 922.147
36395 || 36395
943.4972 || 943.4972


In [26]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)
calculateNormRecall(indexes, trueIndexes)

Recall@1: 0.3766
Recall@1.01: 0.626
Recall@1.1: 0.9716
97
norm Recall is 0.7753


0.7753